In [2]:
%reload_ext autoreload
%autoreload 2

In [3]:
import pandas as pd
from pathlib import Path

from mbp.webscraping import activate_web_driver, get_team_page
from mbp.data import download_team_data, get_team_games_for_year
from mbp.paths import SEASONS_DIR, RAW_DATA_DIR
from mbp.models import TeamYear, TeamGame

In [4]:
team_name = "Arizona"
year = 2022

In [5]:
driver = activate_web_driver("firefox")

# Read team names
df = pd.read_csv(f"{RAW_DATA_DIR}/mbb_team_names_to_number.csv", index_col=0)

In [6]:
games_df = get_team_games_for_year(driver, df, team_name, year)
games_df.head()

/Users/auser/work/aiml/mine/ncaa-mb-prediction/mbp/mbp/webscraping.py:236: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '95' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  games_df2.at[idx, "team_score"] = score_parts[0]
/Users/auser/work/aiml/mine/ncaa-mb-prediction/mbp/mbp/webscraping.py:237: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '78' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  games_df2.at[idx, "opp_score"] = score_parts[1]


,opponent,result,attendance,datetime,win,team_score,opp_score,home
0,Southern U.,95-78,"13,485",2022-11-11,1.0,95,78,1.0
1,Utah Tech,104-77,"12,752",2022-11-17,1.0,104,77,1.0
2,Cincinnati,101-93,"2,400",2022-11-21,1.0,101,93,0.0
3,San Diego St.,87-70,"2,400",2022-11-22,1.0,87,70,0.0
4,Creighton,81-79,"2,400",2022-11-23,1.0,81,79,0.0


In [9]:
# Download Arizona data
download_team_data(team_name, year, True)

/Users/auser/work/aiml/mine/ncaa-mb-prediction/mbp/mbp/webscraping.py:236: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '95' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  games_df2.at[idx, "team_score"] = score_parts[0]
/Users/auser/work/aiml/mine/ncaa-mb-prediction/mbp/mbp/webscraping.py:237: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '78' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  games_df2.at[idx, "opp_score"] = score_parts[1]


In [11]:
# Get their games for the year
year_dir = Path(SEASONS_DIR / str(year))
team_dir = year_dir / team_name
games_df = pd.read_csv(team_dir / "games.csv")

for opp in games_df['opponent']:
    opponent_games = year_dir / opp / "games.csv"
    if not opponent_games.exists():
        print(opp)
        download_team_data(opp, year)

In [12]:
from mbp.models import TeamYear, TeamGame

In [30]:
az = TeamYear("Arizona", 2022)
az.download_updated_data(False)

games = az.get_games()

In [31]:
from datetime import datetime, timedelta, date
today = date.today()
today = today + timedelta(days=-290)
print(today.strftime("%Y-%m-%d"))

df2 = games.copy()
df2['datetime'] = pd.to_datetime(df2['datetime'])
df2['difference'] = (df2['datetime'].dt.date - today)
# next_row = df.loc[df['difference'] == df['difference'].min()]
# print(df2.loc[df2['TimeDifference'].min()])
future_dates = df2[df2['difference'] > timedelta(days=0)]
next_future_date = future_dates.loc[future_dates['difference'].idxmin()]
next_future_date

2023-01-01


opponent               Washington
result                      70-67
attendance                 13,562
datetime      2023-01-05 00:00:00
win                           1.0
team_score                     70
opp_score                      67
home                          1.0
difference        4 days, 0:00:00
Name: 13, dtype: object

In [94]:
from mbp.models import get_next_game

today = date.today()
last_year_date = today + timedelta(days=-170)

az.get_next_opponent_or_last(last_year_date)
game = get_next_game(az, year, last_year_date)
game_stats = game.get_game_stats()
# # game.get_players()
game_stats

,player,pos,g,mp,fgm,fga,3fg,3fga,ft,fta,...,drebs,tot reb,ast,to,stl,blk,fouls,dq,tech fouls,team
0,"Pierce, Caden",F,1,33:49,2,6,,4,2,3,...,6,8,1,2,1,1,4,,,Princeton
1,"Kellman, Keeshawn",F,1,13:41,4,4,,,,,...,3,4,,1,,2,3,,,Princeton
2,"Allocco, Matt",G,1,38:07,3,12,,3,,,...,2,3,2,1,2,,,,,Princeton
3,"Langborg, Ryan",G,1,31:06,4,13,,6,,,...,1,2,1,2,1,2,2,,,Princeton
4,"Evbuomwan, Tosan",F,1,36:23,7,15,,1,1,2,...,7,7,4,2,1,1,3,,,Princeton
5,"Peters, Blake",G,1,15:16,3,6,3,5,,,...,3,3,,1,1,,,,,Princeton
6,"Lee, Xaivian",G,1,2:14,,1,,1,,,...,1,1,,1,,,1,,,Princeton
7,"Martini, Zach",F,1,29:24,3,7,1,5,,,...,5,7,1,1,2,,2,,,Princeton
8,TEAM,,,,,,,,,,...,1,3,,,,,,,,Princeton
9,"Tubelis, Azuolas",F,1,37:18,9,20,1,2,3,4,...,4,5,1,6,4,1,4,,,Arizona


AttributeError: 'TeamGame' object has no attribute 'game_stats'

In [98]:
team_a = game_stats.loc[game_stats['team'] == game.team_a.team_name]
all_team_a_players = game.team_a.get_roster()
player_names = [p for p in all_team_a_players['player']]
active_players = [p for p in team_a['player'] if p in player_names]
active_players

['Tubelis, Azuolas',
 'Henderson Jr., Cedric',
 'Ramey, Courtney',
 'Kriisa, Kerr',
 'Ballo, Oumar',
 'Veesaar, Henri',
 'Boswell, Kylan',
 'Larsson, Pelle']

In [117]:
stats = game.team_a.get_stats()
team_avg = 0
df1 = game.team_a.get_roster()
df2 = team_a
team_a_combined = df1.merge(df2, on='player')
team_a_combined['height_ft'].min()
team_a_combined['3fg'][0]

''